In [2]:
import os
from openai import OpenAI
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pprint import pprint
import time
import requests
import os

In [3]:
possible_links = ["https://portal.nccs.nasa.gov/datashare/giss_cmip6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f2/Amon/hus/gn/v20190903/",
                  "https://portal.nccs.nasa.gov/datashare/giss_cmip6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f2/Amon/pr/gn/v20190903/",
                  "https://portal.nccs.nasa.gov/datashare/giss_cmip6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f2/Amon/tas/gn/v20190903/",
                  "https://portal.nccs.nasa.gov/datashare/giss_cmip6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r1i1p1f2/CFday/cct/gn/v20190903/"
                  ]

In [4]:
if not os.environ.get("OPENAI_API_KEY"):
    import key
    key.init()
    assert os.environ.get('OPENAI_API_KEY')
    
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [5]:
def create_file(path="pr_Amon_GISS-E2-1-G_ssp245_r10i1p1f2_gn_201501-205012.nc"):
    file = client.files.create(
        file=open(path, "rb"),
        purpose='assistants'
    )
    return file


def create_assistant(file):
    assistant = client.beta.assistants.create(
        name="Climate PAL Assistant",
        instructions="You are a climate scientist that is an expert in analyzing and plotting data. When asked a climate science question, write and run code to answer the question. Use the xarray library to read .nc files. Download and use the cftime library to decode time variables",
        tools=[{"type": "code_interpreter"}],
        model="gpt-4o",
        # I need to figure out what this does
        tool_resources={
        "code_interpreter": {
        "file_ids": [file.id]
        }
        }
    )
    return assistant
  
def create_thread(text_body=''):
  thread = client.beta.threads.create()
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = text_body
  )
  print("Here is the message we just created: ", message)
  return thread
  
def create_message(thread, text_body=''):
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = text_body
  )
  print("Here is the message we just created: ", message)
  return message



def download_file(url):
    # Send a HTTP request to the URL
    response = requests.get(url, stream=True)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Extract the filename from the URL
        local_filename = os.path.basename(url)
        
        # Open a local file with write-binary mode
        with open("data/"+local_filename, "wb") as file:
            # Write the contents of the response to the file
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Download completed successfully. File saved as {local_filename}")
    else:
        print("Failed to download the file. Status code:", response.status_code)
        
    return local_filename    

In [6]:
file = create_file()
assistant = create_assistant(file)
thread = create_thread("Create a plot of total rainfall")
create_message(thread, "ignore this sentence, I just want to see how the history works")



Here is the message we just created:  Message(id='msg_vUPeoFPTnbYZ4uqGrtm0etir', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create a plot of total rainfall'), type='text')], created_at=1721724890, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MarBxpendgHjJH7Sc0yHbgUR')
Here is the message we just created:  Message(id='msg_kF1DrG0pRlCA57yu41YHA3ZZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='ignore this sentence, I just want to see how the history works'), type='text')], created_at=1721724891, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MarBxpendgHjJH7Sc0yHbgUR')


Message(id='msg_kF1DrG0pRlCA57yu41YHA3ZZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='ignore this sentence, I just want to see how the history works'), type='text')], created_at=1721724891, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MarBxpendgHjJH7Sc0yHbgUR')

In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [8]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
print(run.status)


in_progress


In [9]:
while run.status not in ["completed", "failed"]:
  run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
  )

  print(run.status)
  time.sleep(5)

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed


In [10]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_sTGOkbncB8CliFlwrrrVVCiB', assistant_id='asst_iau288XZXLFUNy7Am11x6EPF', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I don\'t have internet access to install additional packages in this environment. However, you can install the `cftime` package locally in your environment using the following command:\n\n```sh\npip install cftime\n```\n\nOnce `cftime` is installed, you can run the following code to decode the time variable and plot the total rainfall:\n\n```python\nimport xarray as xr\nimport cftime\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Load the dataset\nfile_path = \'path_to_your_file.nc\'\ndataset = xr.open_dataset(file_path, decode_times=False)\n\n# Extract time variable\ntime_raw = dataset[\'time\'].values\n\n# Decode the time using cftime\nunits = "days since 2015-1-1"\ncalendar = "365_day"\ntime_decoded = cftime.num2date(time_raw, units=units, calendar=calendar)\n\n#

In [11]:
pprint(messages)

SyncCursorPage[Message](data=[Message(id='msg_sTGOkbncB8CliFlwrrrVVCiB', assistant_id='asst_iau288XZXLFUNy7Am11x6EPF', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I don\'t have internet access to install additional packages in this environment. However, you can install the `cftime` package locally in your environment using the following command:\n\n```sh\npip install cftime\n```\n\nOnce `cftime` is installed, you can run the following code to decode the time variable and plot the total rainfall:\n\n```python\nimport xarray as xr\nimport cftime\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Load the dataset\nfile_path = \'path_to_your_file.nc\'\ndataset = xr.open_dataset(file_path, decode_times=False)\n\n# Extract time variable\ntime_raw = dataset[\'time\'].values\n\n# Decode the time using cftime\nunits = "days since 2015-1-1"\ncalendar = "365_day"\ntime_decoded = cftime.num2date(time_raw, units=units, calendar=calendar)\n\n#

In [12]:
for message in reversed(messages.data):
    content_block = message.content[0]
    
    if hasattr(content_block, 'text'):
        print(message.role + ": " + content_block.text.value)
        
    elif hasattr(content_block, 'image_file'):
        # print(content_block.image_file)
        print(message.role + ": [Non-text content]")
        # print(content_block)
        # print(content_block.image_file.file_id)
        
        api_response = client.files.with_raw_response.retrieve_content(content_block.image_file.file_id)

        content = api_response.content
        with open(f"assistant_images/{content_block.image_file.file_id}.png", 'wb') as f:
            f.write(content)
        
        image_path = f"assistant_images/{content_block.image_file.file_id}.png"

        img = mpimg.imread(image_path)
        plt.imshow(img)
        print('File downloaded successfully.')



user: Create a plot of total rainfall
user: ignore this sentence, I just want to see how the history works
assistant: Let's start by investigating the contents of the uploaded file. I'll load it using the `xarray` library and inspect the dataset to identify the variables present.
assistant: It looks like the dataset uses a non-standard calendar ('365_day'), which cannot be decoded by pandas. I will try opening the dataset with the `decode_times=False` option and explore the variables. Then, I can manually decode the time variable using the `cftime` library if needed.

Let's proceed with that.
assistant: The dataset contains variables including `pr` (which likely stands for precipitation), as well as coordinates for `time`, `lat`, and `lon`. The time variable needs manual decoding.

I'll decode the time variable using cftime, and then proceed to calculate the total rainfall by summing the precipitation values over the entire time period. Finally, I'll create a plot of the total rainfall

In [12]:
response = client.beta.assistants.delete(assistant.id)
print(response)

response = client.files.delete(file)
print(response)

AssistantDeleted(id='asst_WMtCqvsSAdRXoymTW8ODhk2g', deleted=True, object='assistant.deleted')


NotFoundError: Error code: 404 - {'error': {'message': "No such File object: FileObject(id='file-HRuRVNU56PuXGRztQDVw0i5m', bytes=22424788, created_at=1721297692, filename='pr_Amon_GISS-E2-1-G_ssp245_r10i1p1f2_gn_201501-205012.nc', object='file', purpose='assistants', status='processed', status_details=None)", 'type': 'invalid_request_error', 'param': 'id', 'code': None}}